# Capsule Defect Detection System

## Overview

The Capsule Defect Detection System is a real-time machine vision application designed for automated quality control in manufacturing environments. Using a Basler camera interfaced through the pypylon library, the system captures images of capsules on a moving conveyor belt, processes these images with OpenCV-based algorithms, and detects potential defects. When an abnormal capsule is identified, the system triggers a relay to take appropriate action — such as diverting the defective item.

## Features

- **Real-Time Image Acquisition:**  
  Leverages pypylon to continuously capture frames from an industrial camera.

- **Advanced Image Processing:**  
  Utilizes OpenCV and custom algorithms to preprocess images, extract contours, and analyze capsule characteristics.

- **Defect Detection:**  
  Implements specialized logic to differentiate between normal and abnormal capsules based on metrics such as size, area, and similarity.

- **Hardware Integration:**  
  Controls external hardware (relay) to automate responses based on defect detection outcomes.

- **Debugging and Visualization:**  
  Provides real-time visualization and logging to facilitate debugging and performance monitoring during development.

## Architecture

The project is structured into several modular components:

- **Image Acquisition:**  
  Uses the pypylon library to initialize the camera, set exposure and frame rate, and capture images continuously.

- **Image Processing & Analysis:**  
  - **Preprocessing:** Functions in `utils.transform` prepare the raw image data.  
  - **Contour Extraction:** The `src.contours` module extracts potential capsule boundaries.  
  - **Defect Detection:** The `src.defects` module analyzes the extracted contours to identify abnormal capsules based on predefined criteria.

- **Hardware Control:**  
  The `src.relay_controller` module manages relay operations, turning the relay on or off based on the timing computed from the capsule positions and conveyor belt parameters.

- **Configuration & Parameters:**  
  The `src.params` file contains critical configuration values such as belt speed, camera settings, and physical dimensions necessary for accurate image analysis and actuation timing.

## Project Structure
```text
project/
├── data/
│   └── Figs_14/
│       └── 000_mask_raw.png
├── src/
│   ├── contours.py
│   ├── defects.py
│   ├── params.py
│   └── relay_controller.py
├── utils/
│   ├── transform.py
│   └── visualize.py
├── notebooks/
│   └── Capsule_Defect_Detection.ipynb
└── main.py
```

In [1]:
# Import standard libraries
import sys
import logging
import time
from collections import deque

# Import third-party libraries
import cv2
import numpy as np
from pypylon import pylon

from PyQt5.QtCore import QThread, pyqtSignal, Qt
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtWidgets import QApplication, QLabel, QVBoxLayout, QWidget, QMainWindow

# Import project functions and parameters
from src.contours import find_contours_img
from src.defects import detect_capsule_defects
from src.params import (
    BELT_LENGTH_MM, BELT_SPEED_MM_S, GRABBING_TIMEOUT_MS,
    INIT_EXPOSURE_TIME, INIT_FRAME_RATE, INIT_GAIN,
    INIT_HEIGHT, INIT_WIDTH, MM_PER_PIXEL, RELAY_1
)
from src.relay_controller import RelayController
from utils.transform import get_img_opened
from utils.visualize import cvimshow

# Set debug flag and logging level
DEBUG_MODE = True
logging.basicConfig(
    level=logging.DEBUG if DEBUG_MODE else logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

# Load and preprocess the mask image
MASK_IMG_PATH = "./data/Figs_14/000_mask_raw.png"
MASK_RAW = cv2.imread(MASK_IMG_PATH)
MASK_BIN = get_img_opened(MASK_RAW)

In [2]:
def initialize_camera() -> pylon.InstantCamera:
    """Initialize the camera and configure its settings."""
    try:
        camera = pylon.InstantCamera(
            pylon.TlFactory.GetInstance().CreateFirstDevice()
        )
        logging.debug("Connected to camera: %s",
                      camera.GetDeviceInfo().GetFriendlyName())
    except (pylon.GenericException, RuntimeError) as e:
        logging.error("Error retrieving camera: %s", e)
        return

    camera.Open()

    # Configure camera settings
    camera.ExposureTime.Value = INIT_EXPOSURE_TIME
    camera.AcquisitionFrameRateEnable.Value = True
    camera.AcquisitionFrameRate.SetValue(INIT_FRAME_RATE)
    camera.AcquisitionMode.SetValue("Continuous")
    camera.Gain.SetValue(INIT_GAIN)
    camera.Width.SetValue(INIT_WIDTH)
    camera.Height.SetValue(INIT_HEIGHT)

    # Center the camera's region of interest
    camera.OffsetX.Value = (camera.Width.GetMax() - INIT_WIDTH) // 2
    camera.OffsetY.Value = (camera.Height.GetMax() - INIT_HEIGHT) // 2

    # Log sensor dimensions (converted from µm to mm)
    sensor_width_mm = camera.SensorWidth.GetValue() / 1000.0
    sensor_height_mm = camera.SensorHeight.GetValue() / 1000.0
    logging.debug("Sensor Size: %.2f mm x %.2f mm",
                  sensor_width_mm, sensor_height_mm)

    # Prepare the camera grabbing mode and image converter
    camera.StopGrabbing()
    camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)

    return camera

In [3]:
converter: pylon.ImageFormatConverter = pylon.ImageFormatConverter()
# Converting to opencv bgr format
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned

# shape = (nrows, ncols)
# Explicitly declaring an array helps reducing processing time in blocking polling loop
grab_result: pylon.GrabResult
pylon_image: pylon.PylonImage
image: np.ndarray[np.uint8]
capsule_centers_abnormal: list[tuple[int, int]]

# Max length of the double-ended queue should be larger than the number of capsules
# that could possibly be detected in the field-of-view in 1 frame
# Adding timestamps with append() automatically removes oldest when full
abs_actuation_time_stamps: deque[float] = deque(maxlen=20)
abs_actuation_timestamps: list[float] = []

In [4]:
relay_controller: RelayController = RelayController()

2025-02-27 10:17:58,306 - DEBUG - Device found: DEVICE ID 16c0:05df on Bus 000 Address 001 =================
 bLength                :   0x12 (18 bytes)
 bDescriptorType        :    0x1 Device
 bcdUSB                 :  0x110 USB 1.1
 bDeviceClass           :    0x0 Specified at interface
 bDeviceSubClass        :    0x0
 bDeviceProtocol        :    0x0
 bMaxPacketSize0        :    0x8 (8 bytes)
 idVendor               : 0x16c0
 idProduct              : 0x05df
 bcdDevice              :  0x100 Device 1.0
 iManufacturer          :    0x1 www.dcttech.com
 iProduct               :    0x2 USBRelay4
 iSerialNumber          :    0x0 
 bNumConfigurations     :    0x1
  CONFIGURATION 1: 20 mA ===================================
   bLength              :    0x9 (9 bytes)
   bDescriptorType      :    0x2 Configuration
   wTotalLength         :   0x22 (34 bytes)
   bNumInterfaces       :    0x1
   bConfigurationValue  :    0x1
   iConfiguration       :    0x0 
   bmAttributes         :   0x80 Bus 

In [5]:
class CameraThread(QThread):
    # Signal to send the frame image, frame count, and timestamp to the UI
    frame_signal = pyqtSignal(np.ndarray, int, float)  # image, frame count, timestamp

    def run(self):
        try:
            camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        except Exception as e:
            print("Error initializing camera:", e)
            return

        camera.Open()
        # Set camera parameters
        camera.ExposureTime.Value = INIT_EXPOSURE_TIME
        camera.AcquisitionFrameRate.SetValue(INIT_FRAME_RATE)
        camera.Gain.SetValue(INIT_GAIN)
        camera.Width.SetValue(INIT_WIDTH)
        camera.Height.SetValue(INIT_HEIGHT)
        camera.StopGrabbing()

        # Start grabbing using the latest image strategy
        camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)

        count = 0
        while camera.IsGrabbing() and not self.isInterruptionRequested():
            try:
                grab_result = camera.RetrieveResult(GRABBING_TIMEOUT_MS, pylon.TimeoutHandling_ThrowException)
            except Exception as e:
                print("Error retrieving frame:", e)
                break

            if grab_result.GrabSucceeded():
                image = grab_result.Array  # Should be a 2D (grayscale) image
                grab_time = time.time()
                self.frame_signal.emit(image, count, grab_time)
                count += 1

            grab_result.Release()

        camera.StopGrabbing()
        camera.Close()

In [ ]:
class CameraWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Camera Feed")

        # Create a label for the image and one for execution status
        self.image_label = QLabel("Waiting for image...")
        self.status_label = QLabel("Status: Starting")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.status_label.setAlignment(Qt.AlignCenter)

        # Use a vertical layout to stack the labels
        central_widget = QWidget()
        layout = QVBoxLayout(central_widget)
        layout.addWidget(self.image_label)
        layout.addWidget(self.status_label)
        self.setCentralWidget(central_widget)

        # Start the camera thread and connect its signal to the update function
        self.camera_thread = CameraThread()
        self.camera_thread.frame_signal.connect(self.update_frame)
        self.camera_thread.start()

    def update_frame(self, image: np.ndarray, count: int, timestamp: float):
        # Convert the grayscale NumPy array to a QImage.
        # The image is assumed to be 2D with dimensions (INIT_HEIGHT, INIT_WIDTH)
        height, width = image.shape
        bytes_per_line = width
        qimg = QImage(image.data, width, height, bytes_per_line, QImage.Format_Grayscale8)
        pixmap = QPixmap.fromImage(qimg)

        # Scale the pixmap to fit the label while preserving aspect ratio
        self.image_label.setPixmap(pixmap.scaled(self.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))

        # Update the status label with frame count and timestamp
        status_text = f"Frame: {count} | Timestamp: {timestamp:.2f}"
        self.status_label.setText(status_text)

    def resizeEvent(self, event):
        # Ensure the image is rescaled on window resize
        if self.image_label.pixmap():
            self.image_label.setPixmap(self.image_label.pixmap().scaled(self.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        super().resizeEvent(event)

    def closeEvent(self, event):
        # Request the camera thread to stop and wait for it to finish when closing the window
        self.camera_thread.requestInterruption()
        self.camera_thread.wait()
        event.accept()

In [7]:
def main():
    app = QApplication(sys.argv)
    window = CameraWindow()
    window.resize(800, 600)
    window.show()
    sys.exit(app.exec())

if __name__ == "__main__":
    main()

SystemExit: 0

C:\Users\hux1mis\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
